In [1]:
import sys
sys.path.append('../')
import numpy as np
import matplotlib.pyplot as plt
import functions
from pprint import pprint
import json
from scipy import interpolate
import langmuir.gymness.aggregate as agg
from decimal import Decimal
import pandas as pd
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
# definisco i file
lamp_file = '../OES_Studenti_2017/Ilamp_acquisitiontime_3s.txt'
rho_file = '../OES_Studenti_2017/riflettivitadiffusore.txt' 
argon_750_file = '../OES_Studenti_2017/Ar15022017_a8_acq_time_0.01s.txt' 
argon_480_file = '../OES_Studenti_2017/Ar15022017_a1_acq_time_0.5s.txt' 

acquisition_time = 0.01

peak_ratio = functions.get_peak_ratio(argon_480_file)
[I_abs_spectrum, n_0, I_abs_750, X_750] = functions.get_absolute_intensity(lamp_file, rho_file, argon_750_file, acquisition_time)
n_e_mean, n_e_min, n_e_max = functions.get_density(peak_ratio, '../fantz-data/fants.txt')

In [3]:
# peak_ratio_approx = '%.3E' % peak_ratio
# n_0_approx = '%.3E' % n_0
# I_abs_750_approx = '%.3E' % I_abs_750
# X_750_approx = '%.3E' % X_750
# n_e_mean_approx = '%.3E' % n_e_mean
# n_e_min_approx = '%.3E' % n_e_min
# n_e_max_approx = '%.3E' % n_e_max
print("""
Peak ratio I480/I488 = {:.3E}
n0 = {:.3E} cm-3
I absolute 750 = {:.3E} 
X 750 = {:.3E}
n_e_mean = {:.3E} cm-3
( {:.3E} < n_e < {:.3E} ) cm-3""". format(peak_ratio, n_0, I_abs_750, X_750, n_e_mean, n_e_min, n_e_max))


Peak ratio I480/I488 = 5.252E-01
n0 = 1.204E+18 cm-3
I absolute 750 = 1.986E-13 
X 750 = 4.782E+16
n_e_mean = 1.200E+11 cm-3
( 1.072E+11 < n_e < 1.338E+11 ) cm-3


In [4]:
# density ratio pression spectra analysis
#                                                    name of config file    folder with data
ratios_p1, w_em_p1, results_p1 = functions.get_profile_ratio('../config_p1_2302.json', '../data/2302/2302spectr/')
ratios_p2, w_em_p2, results_p2 = functions.get_profile_ratio('../config_p2_2302.json', '../data/2302/2302spectr/')
n_avg_p1, n_min_p1, n_max_p1  = functions.get_density(ratios_p1, '../fantz-data/fants.txt')
n_avg_p2, n_min_p2, n_max_p2  = functions.get_density(ratios_p2, '../fantz-data/fants.txt')
p1_off = 3.1e-5
p2_off = 1.74e-4

In [5]:
# create a pandas dataframe for densities
df_spectra_n = pd.DataFrame()
# for p1
for w, content in results_p1.items():
    df = pd.read_json(json.dumps(content), orient="records")
    df['w'] = w
    df['p_off'] = p1_off
    df_spectra_n = pd.concat([df_spectra_n, df])
    
# for p2
for w, content in results_p2.items():
    df = pd.read_json(json.dumps(content), orient="records")
    df['w'] = w
    df['p_off'] = p2_off
    df_spectra_n = pd.concat([df_spectra_n, df])

# apply the get density function to every row and save the results in a density dataframe
df = pd.DataFrame(df_spectra_n['ratio'].apply(functions.get_density).tolist(), columns=['n', 'n_min', 'n_max'])
df_spectra_n = df_spectra_n.reset_index()
del df_spectra_n['index']

# concatenate the two dataframes
df_spectra_n = pd.concat([df_spectra_n, df], axis=1)
# group by power and take average
a = df_spectra_n.groupby(['p_off', 'w', 'id']).mean()
a

p     ratio             n         n_min  \
p_off    w  id                                                        
0.000031 20 a10ad10  0.000050  0.475925  1.079281e+11  9.396793e+10   
            a12ad12  0.000050  0.461439  1.037000e+11  8.775966e+10   
            a13ad13  0.000050  0.454845  1.016664e+11  8.493361e+10   
            a14ad14  0.000050  0.452504  1.009446e+11  8.393046e+10   
         40 a1ad1    0.000085  0.480338  1.091107e+11  9.585906e+10   
            a3ad3    0.000085  0.470204  1.063948e+11  9.151617e+10   
            a7ad7    0.000085  0.471929  1.068571e+11  9.225542e+10   
         60 a18ad18  0.000040  0.557539  1.276359e+11  1.137835e+11   
            a20ad20  0.000040  0.534847  1.222818e+11  1.091525e+11   
            a21ad21  0.000040  0.536467  1.226640e+11  1.094831e+11   
            a22ad22  0.000040  0.536723  1.227245e+11  1.095354e+11   
0.000174 20 a26ad26  0.000189  0.482237  1.096197e+11  9.667289e+10   
            a28ad28  0.000189  0.469045  1.060455e+11  9.101921e+10   
            a29ad29  0.000189  0.464108  1.045231e+11  8.890354e+10   
            a30ad30  0.000189  0.475004  1.076813e+11  9.357329e+10   
         40 a34ad34  0.000200  0.521656  1.191694e+11  1.064604e+11   
            a36ad36  0.000200  0.505165  1.152783e+11  1.030948e+11   
            a37ad37  0.000200  0.501485  1.144102e+11  1.023440e+11   
            a38ad38  0.000200  0.499693  1.139874e+11  1.019782e+11   
         60 a42ad42  0.000201  0.525343  1.200392e+11  1.072128e+11   
            a44ad44  0.000201  0.505942  1.154617e+11  1.032534e+11   
            a45ad45  0.000201  0.505853  1.154407e+11  1.032353e+11   
            a46ad46  0.000201  0.507210  1.157609e+11  1.035122e+11   

                            n_max  
p_off    w  id                     
0.000031 20 a10ad10  1.205203e+11  
            a12ad12  1.166052e+11  
            a13ad13  1.148230e+11  
            a14ad14  1.141904e+11  
         40 a1ad1    1.217129e+11  
            a3ad3    1.189742e+11  
            a7ad7    1.194404e+11  
         60 a18ad18  1.425782e+11  
            a20ad20  1.364452e+11  
            a21ad21  1.368830e+11  
            a22ad22  1.369523e+11  
0.000174 20 a26ad26  1.222261e+11  
            a28ad28  1.186608e+11  
            a29ad29  1.173266e+11  
            a30ad30  1.202714e+11  
         40 a34ad34  1.328800e+11  
            a36ad36  1.284229e+11  
            a37ad37  1.274285e+11  
            a38ad38  1.269441e+11  
         60 a42ad42  1.338764e+11  
            a44ad44  1.286329e+11  
            a45ad45  1.286089e+11  
            a46ad46  1.289756e+11

In [6]:
# create pandas dataframe for temperatures
# open the json file
df_spectra_T = pd.DataFrame()
with open('../config_p1_2302.json') as file:
    data = json.load(file)
    df = pd.read_json(json.dumps(data['T']), orient='index')
    df['p_off'] = p1_off
    df_spectra_T = pd.concat([df_spectra_T, df])

with open('../config_p2_2302.json') as file:
    data = json.load(file)
    df = pd.read_json(json.dumps(data['T']), orient='index')
    df['p_off'] = p2_off
    df_spectra_T = pd.concat([df_spectra_T, df])
    
df_spectra_T = df_spectra_T.reset_index()
df_spectra_T = df_spectra_T.rename(columns={'index': 'id'})

folder_path = '../data/2302/2302spectr/'
extension = '.dat'

df_spectra_T['file'] = folder_path + df_spectra_T['id'] + extension
df_spectra_T['wavelength'] = 750.4
df_spectra_T['i_abs'] = df_spectra_T.apply(lambda x: functions.get_absolute_intensity_2(x['file'],x['t'], x['wavelength']), axis=1)



In [7]:
# time to shoot myself
df = pd.merge(df_spectra_T, df_spectra_n, on=['p', 'w', 'p_off'])
df['p_correction'] = 1/1.29
a = df.groupby(['p', 'w'], as_index=False).mean()
a['pec'] = a.apply(lambda x: functions.get_pec(x['i_abs'], x['p'], x['p_correction'], x['n']), axis=1)

# adas reading and interpolation
df_adas = functions.read_adas_file('../data/adas/ar_750.dat')
mesh_n, mesh_pec = np.meshgrid(np.log10(a['n']), np.log10(a['pec']))
a['T'] = functions.interpolate_dataframe(df_adas, (np.log10(a['n']), np.log10(a['pec'])))
df = a
df

,p,w,t,p_off,wavelength,i_abs,ratio,n,n_min,n_max,p_correction,pec,T
0,0.000040,60,0.01,0.000031,750.4,2.281134e+08,0.541394,1.238265e+11,1.104886e+11,1.382147e+11,0.775194,2.449103e-14,9.281740
1,0.000050,20,0.01,0.000031,750.4,1.045433e+08,0.456263,1.021037e+11,8.554125e+10,1.152062e+11,0.775194,1.086245e-14,12.788578
2,0.000085,40,0.01,0.000031,750.4,1.235702e+08,0.474157,1.074542e+11,9.321022e+10,1.200425e+11,0.775194,7.176523e-15,13.155719
3,0.000189,20,0.01,0.000174,750.4,3.182297e+08,0.472599,1.069674e+11,9.254223e+10,1.196212e+11,0.775194,8.349693e-15,13.005149
4,0.000200,40,0.01,0.000174,750.4,5.133075e+08,0.507000,1.157113e+11,1.034693e+11,1.289188e+11,0.775194,1.176561e-14,11.937216
5,0.000201,60,0.01,0.000174,750.4,5.731063e+08,0.511087,1.166756e+11,1.043034e+11,1.300234e+11,0.775194,1.296289e-14,11.669314


In [ ]:
# plotting data
fig_spectr, ax_spectr = plt.subplots(1,1)
ax_spectr.errorbar(w_em_p1, n_avg_p1, yerr=n_max_p1-n_min_p1, fmt='o', label='p = {} mbar'.format(p1_off))
ax_spectr.errorbar(w_em_p2, n_avg_p2, yerr=n_max_p2-n_min_p2, fmt='o', label='p = {} mbar'.format(p2_off))
ax_spectr.legend()
ax_spectr.grid()
# second pression spectra analysis

In [ ]:
# langmuir density and profile temperature
#                        path to results data
df = agg.aggregate_data('../langmuir/data/data-2302')
# the grouped by columns will become indexes
a = df.groupby(['P_off', 'W_em'])['n_e', 'err_n_e', 'T_e', 'err_T_e'].mean()
# set plot stles, titles and figures. Need to add the labels though
fig = plt.figure()
ax_ne = fig.add_subplot(211)
ax_ne.set_title('$n_e$ $cm^{-3}$')
ax_ne.grid()
ax_te = fig.add_subplot(212)
ax_te.set_title('$T_e$ (eV)')
ax_te.grid()
plt.tight_layout()  # this is useful for overlapping figures

# difficult code: iterate over the values of the pressure
# indexes (there are just two values in this case) and
# take the values of the w, n, and T for each p.
# Then plot all the data with their labels
for p in a.index.get_level_values(0).unique():
    w = a.index.get_level_values(1).unique()
    n_e = a.loc[p,:]['n_e']
    err_n_e = a.loc[p,:]['err_n_e']
    T_e = a.loc[p,:]['T_e']
    err_T_e = a.loc[p,:]['err_T_e']
    ax_ne.errorbar(w, n_e, yerr=err_n_e, fmt='o', label="p: {} mbar".format(p))
    ax_te.errorbar(w, T_e, yerr=err_T_e, fmt='o', label="p: {} mbar".format(p))
    
# set legend for the plots
ax_ne.legend()
ax_te.legend()